## TL;DR: Jumble solver using a hash table and letter-wise sorting of all english words. `O(n)` time and space complexity in the average case but with room for optemization both algorithmically/mathematically and infrastructurally.

As I understand the prompt, the task is to write a Jumble solver. It'll take a static word list and a single word user input. The solver will print out any valid words from the list that can be constructed from any arrangement of some or all the letters in the user input, except the original word.

Let's load a dictionary into memory as an example word list.

In [2]:
def load_words():
    with open('words_alpha.txt') as word_file:
        valid_words = word_file.read().split()
    return valid_words


english_words = load_words()
print("There are " + str(len(english_words)) + " words in this dictionary.")

There are 370103 words in this dictionary.


With 370103 words in this `english_words` array, we'll have to be clever about scanning through it to find matches to the user input. We can probably do this best by not scanning through it over and over haha. Maybe there's some super clever dynamic programming way to do this or something like that, but this is how I think about this problem: the order of the letters in a word is irrelevant in this problem except when printing the results, and it is this order which makes the problem difficult to solve as is, so why not get rid of it? With a list of sorted words, it becomes much easier to take the `user_input` (and every possible substring thereof), sort its letters, and then find a match in the `english_words` list. There are some complications though.

If we sort the letters in each word in `english_words`, we'll lose the ability to sensibly print out matches, so we'll have to keep some kind of link between the sorted and unsorted versions of each word in the list, which feels like a lookup table (called `sorted_words_table`) to me. This is to our advantage since a hash table solution to this issue also speeds up process of looking up the sorted substrings of `user_input`.

So here's the process: loop through `english_words`, sorting each word from a-z (assuming for now that a-z is the only thing we'll see, since that's the case for this word list, but since they'll be sorted by ASCII codes, it doesn't matter), and looking them up in a dict called `sorted_words_table` as we go. If there's no entry in `sorted_words_table` at that location, then add the index of the unsorted version of the current word in `english_words` to an array and stick that in the hash table at that location. If such an array already exists, that means we've found an annagram which will need to be printed out as well if a substring of the `user_input` matches this location in `sorted_words_table`. Then generate all substrings of `user_input` except `user_input` itself, sorting them letter-wise as we go, and looking up their sorted version in `sorted_words_table` and adding any indexes found to a new array called `match_idxs`. Finally, print out the words from `english_words` corresponding to each index in `match_idxs`.

Runtime analysis:
If `n` is the number of words in the list and `m` is the number of words in the inputted word (and, we can safely assume given the context of this game, that of the words in the list, on average), then:
* Generating the `sorted_words_table` is supposed to be `O(n)` since lookup is supposed to be `O(1)` for hash tables and we're doing `n` lookups, but it's complicated -- `O(1)` is not guarenteed in most cases IIRC. IRL, you can use the fact that you know the length of the list ahead of time to allocate enough space for the hash table to not need to grow each time it runs out of space. Also, you could use the fact that you know what kind of data is in the list to design a better hash function. I'm no theorist though and I'd have to play with and research it further. Anyway, since we're also sorting each string as we go, that'd be `O(n * m * log(m))`. Let's go with `O(n)` for the hashing, so together that'd be `O(n) + O(n * m * log(m))`, but I'd say the length of the average english word is sufficiently shorter than the number of english words so that boils down to `O(n)` time complexity and the same for space. `O(n^2)` in the worst case, but this can likely be addressed as mentioned.
* Generating the substrings should be `O(m^2)`, plus `O(m * log(m))` to sort each one, but as stated previously, m is, I think, so much smaller than n as to make the point moot. Therefore, `O(1)` for this step.
* Checking the substrings against the table is at worst `O(m^2*n)` since, again, hash table lookups are `O(n)` in the worst case, but again, this boils down to an `O(n)` worst case because `n >> m`, and the first step we already know takes longer than this so this can also be ignored (at this scale).

Overall runtime is `O(n)` average case, `O(n^2)` worst case.

In [13]:
def get_sorted_words_table(english_words):
    sorted_words_table = dict()
    for idx, word in enumerate(english_words):
        word = "".join(sorted(word))
        if word in sorted_words_table:
            sorted_words_table[word].append(idx)
        else:
            sorted_words_table[word] = [idx]
    return sorted_words_table

sorted_words_table = get_sorted_words_table(english_words)
print("There are " + str(len(sorted_words_table)) + " unique sorted words in this dictionary. That's only slightly less than the original total!")

There are 326947 unique sorted words in this dictionary. That's only slightly less than the original total. 


In [14]:
def get_all_unique_alphabetical_substrings(word):
    all_substrings = [word[i: j] for i in range(len(word)) for j in range(i + 1, len(word) + 1)]
    # can't think of a way to generate all substring except the original in one line
    all_substrings.remove(word)
    return set(["".join(sorted(substring)) for substring in all_substrings])

print(get_all_unique_alphabetical_substrings('test'))

{'t', 's', 'es', 'est', 'e', 'et', 'st'}


In [18]:
# sandbox with user input
def jumble(word):
    all_unique_alphabetical_substrings = get_all_unique_alphabetical_substrings(word)
    match_idxs = []
    for substr in all_unique_alphabetical_substrings:
        if substr in sorted_words_table:
            match_idxs += sorted_words_table[substr]
    return [english_words[idx] for idx in match_idxs]

print(jumble(input()))

['o', 'finis', 'it', 'ti', 'iota', 'tiao', 'nest', 'nets', 'sent', 'sten', 'tens', 'nicotia', 'caitif', 'niota', 'taino', 'nt', 'tn', 'innet', 'tinne', 'ficins', 'en', 'ne', 'net', 'ten', 'a', 'e', 'action', 'atonic', 'cation', 'fines', 'neifs', 'ifs', 'cai', 'cia', 'intines', 'is', 'si', 'int', 'nit', 'tin', 'ns', 'sn', 'ot', 'to', 'finniest', 'in', 'ni', 'ai', 'ia', 'coati', 'ins', 'isn', 'nis', 'sin', 'f', 's', 'inset', 'neist', 'senit', 'senti', 'snite', 'stein', 'stine', 'tiens', 'tines', 'tsine', 'ic', 'finites', 'fisetin', 'nifties', 'nies', 'sine', 'fi', 'if', 'infinites', 'ito', 'toi', 'et', 'te', 'into', 'nito', 'oint', 'tino', 'feints', 'festin', 'finest', 'infest', 'n', 'no', 'on', 'fins', 'iten', 'neti', 'tien', 'tine', 't', 'ens', 'sen', 'ac', 'ca', 'ait', 'ati', 'ita', 'tai', 'i', 'fica', 'cfi', 'cif', 'c', 'not', 'ont', 'ton', 'innest', 'intens', 'sennit', 'sinnet', 'tennis']


I was confused at first because this output is much longer than that in the prompt, but I've verified that the algorithm works as expected -- this word list is just much more complete than I realized. When I enter "dog", I wouldn't have thought "g" was a word, but it's there at index 209,855. What about "od", surely that's not a word you'd say out loud? no, but if you look at the list from which this one was generated, you'll see "O.D." (as in, "overdose") there. Strange, but hey, that's coding, right? Computers do exactly as we say, not as we wish. If I were doing this for a customer, I'd verify ahead of time what kind of dictionary they want to play with (and why they're paying me to do it instead of using something that's already out there?? haha).

Speaking of customers and what they want, what if we wanted to scale this up? Vertical scaling can only get you so far, and impossible though it may seem, if the word list got big enough or sufficiently frequently accessed, I'd want to parallelize the work done by . Early in my python career, I read [this](https://drive.google.com/file/d/0B8euleTxnfmyVTYwWHE2b3ZmdEk/view?usp=sharing&resourcekey=0-4v2E7UunlgpsVkUz6VCYnQ) presentation on the subject and it rocked my world (along with making JS make way way more sense than it would have otherwise). If you'd like to talk about this, lmk ahead of time so I can read back up on it -- it's been long enough that crisp lines of reasoning have blurred into concepts, a map instead of the territory. I could keep up in a conversation about it now, but I'd probably end up using more JS lingo than python haha! But that's my approach to technical converstations anyway -- I don't expect anyone to know everything off the bat but to be able to quickly pick things up. Anyway, thanks for the challenge, talk to you soon!